In [ ]:
import xarray as xr
import rioxarray as rio
import glob
import os
import geopandas as gpd
import hvplot.xarray
import holoviews as hv
hv.extension("bokeh")
import numpy as np

In [ ]:
# Ouverture des données soit depuis plusieurs fichiers soit depuis un seul déjà clip
extent = xr.open_dataarray("/home/onyxia/work/data/masque_etangs.tif").squeeze("band",drop=True)
extent_reduit = xr.open_dataarray("/home/onyxia/work/data/masque_etangs_reduit.tif")
time_serie = xr.open_dataset("/home/onyxia/work/L3/out_dir/S2_L3_2020.nc")
# time_serie_cliped = rio.open_rasterio(r"E:\Stage_PNRBrenne\Data\Data_Sentinel\L3\S2_concat\S2_L3A_2020.nc")

In [ ]:
# Calcul MNDWI
B3 = time_serie["B3"]
B12 = time_serie["B12"]
mndwi = (B3-B12)/(B3+B12)
# mndwi.name = "mndwi_2020"
# path_save = os.path.join("/home/onyxia/work/prod",mndwi.name+".nc")
# mndwi.to_netcdf(path_save)

In [ ]:
resample = mndwi.resample(time="ME").median(dim="time",skipna=True)

In [ ]:
# Application du masque étangs
mndwi_mask = xr.where(extent == 1, resample, np.nan)
mndwi = mndwi_mask

In [ ]:
# Calcul NDVI
B4 = time_serie["B4"]
B8 = time_serie["B8"]
ndvi = (B8-B4)/(B8+B4)
# ndvi.name = "ndvi_2020"
# path_save = os.path.join("/home/onyxia/work/prod",ndvi.name+".nc")
# ndvi.transpose(np.str_('time'), 'y', 'x').rio.to_raster(path_save)
# ndvi.to_netcdf(path_save)

In [ ]:
resample = ndvi.resample(time="ME").median(dim="time",skipna=True)

In [ ]:
# Application du masque étangs
ndvi_mask = xr.where(extent == 1, resample, np.nan)
ndvi = ndvi_mask

In [ ]:
mndwi.name = "mndwi_2020"
ndvi.name = "ndvi_2020"
path_save_m = os.path.join("/home/onyxia/work/prod",mndwi.name+".nc")
mndwi.to_netcdf(path_save_m)
path_save_n = os.path.join("/home/onyxia/work/prod",ndvi.name+".nc")
ndvi.to_netcdf(path_save_n)